### Introduction
This notebook aims to discover some potential patterns by analyzing the content of each post with the help of Apriori algorithm.

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import re

In [2]:
class apriori:
    def __init__(self, min_support, min_confidence):
        self.sets_count = dict()
        self.min_support_sets = None
        self.min_support = min_support
        self.min_confidence = min_confidence
    
    def get_set_count(self, data, set_candidate):
        '''
        Return number of occurrence of set_candidate in the data

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        set_candidate: Set
            A set of items to be counted

        Returns
        ----------
        Int:
            Number of set_candidate in data
        '''  
        item_key = tuple(set_candidate)
        if item_key not in self.sets_count:
            item_count = 0
            for record in data:
                if set_candidate.issubset(record):
                    item_count += 1
            self.sets_count[item_key] = item_count
        return self.sets_count[item_key]
    
    def gen_first_sets(self, data):
        '''
        Return lists of candidate sets. Each set has one item

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        List:
            A list of sets with length one
        '''  
        res = set()
        for record in data:
            res.update(record)
        res = [{i} for i in res]
        return res
    
    def gen_CTable(self, data, subsets):
        '''
        Return support_value for each item in subsets which is greater than min_support

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        subsets: List
            A list of items that need to be counted. Each item should a set

        Returns
        ----------
        Dict: 
            A dictionary mapping items in subsets and corresponding support value.
            format: {tuple : float}
        '''     
        CTable = defaultdict(lambda: 0)
        for item in subsets:
            CTable[tuple(item)] = self.get_set_count(data, item)
        l = len(data)
        for k in list(CTable.keys()):
            if CTable[k]/l < self.min_support:
                CTable.pop(k)
            else:
                CTable[k] /= l
        return CTable
    
    def is_super_set(self, candidate_set, recorded_sets):
        '''
        If any subset of candidate_set is in recorded_sets, return True.
        e.g. candidate_set = {1,2,3}   recorded_sets = {(5,7), (9,), (1,3)}   Return: True 
        Because {1,3} is one of subset of {1,2,3} and it is in recorded_sets

        Parameters
        -----------
        candidate_set: set
            The set to be tested
        
        recorded_sets: Set
            Set of tuples

        Returns
        ----------
        Bool:
            Whether candidate_set is super set of any set in recorded_sets
        '''
        l = len(candidate_set)
        for k in range(1, l):
            sub_candidates = list(itertools.combinations(candidate_set, k))
            for sub_candidate in sub_candidates:
                if sub_candidate in recorded_sets:
                    return True
        return False
 
    
    def gen_k_sets(self, prev_candidates, k):
        '''
        Return lists of candidate sets. Each set has k items

        Parameters
        -----------
        prev_candidates: set
            Set of tuples of satisfying min_support
        
        k: int
            Specify the length of sets generated

        Returns
        ----------
        List:
            A list of sets with length k
        '''
        c_prev = [set(i) for i in prev_candidates]  # convert to list of sets
        c_next = list(itertools.combinations(c_prev, 2)) # list of tuples of 2 sets
        c_next = {tuple(set1.union(set2)) for set1, set2 in c_next if len(tuple(set1.union(set2))) == k}  # convert to set of tuples with length k
        # all subsets with length k-1 of candidate should fulfill min_support
        remove_list = []
        for candidate in c_next:
            sub_candidates = itertools.combinations(candidate, k-1)
            for sub_candidate in sub_candidates:
                if sub_candidate not in prev_candidates:
                    remove_list.append(candidate)
                    break
        for candidate in remove_list:
            c_next.remove(candidate)
        c_next = [set(i) for i in c_next]
        return c_next
    
    def get_min_support_sets(self, data):
        '''
        Return subsets with support greater than min_support

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        Dict:
            A dictionary mapping subsets and corresponding support values
        '''
        # Generate count table for set with length 1
        c_dict = dict()    # c_dict[k] store the CTable of length k
        subsets = self.gen_first_sets(data)
        ck = self.gen_CTable(data, subsets)
        c_dict[1] = ck
        k=2
        while len(c_dict[k-1]) > 1:
            subsets = self.gen_k_sets(set(c_dict[k-1].keys()), k)
            ck = self.gen_CTable(data, subsets)
            c_dict[k] = ck
            k+=1
        self.min_support_sets = c_dict
        return self.min_support_sets
    
    def get_confidence(self, data, left_set, right_set):
        '''
        Return confidence value for the rule 'left_set -> right_set'

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        left_set: set
            A set of items on left hand side of rule
        
        right_set: set
            A set of items on right hand side of rule

        Returns
        ----------
        Float:
            Confidence value of the rule
        '''
        left_count = self.get_set_count(data, left_set)
        combined_count = self.get_set_count(data, left_set.union(right_set))
        return combined_count/left_count
        
        
    def find_rules(self, data):
        '''
        Return rules with confidence value greater than min_confidence

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        Dict:
            A dictionary mapping rules and corresponding confidence values
        '''
        hist = defaultdict(lambda: set())
        res = dict()
        l = len(data)
        if self.min_support_sets is None:
            self.get_min_support_sets(data)
        for k in self.min_support_sets.keys():
            if k == 1:
                continue
            for subset in self.min_support_sets[k].keys():
                for num_left in range(1,k):   # num_left is number of items on left hand side of the rule 
                    num_right = k - num_left
                    left_candidates = list(itertools.combinations(subset, num_left))
                    right_candidates = list(itertools.combinations(subset, num_right))
                    # for each combination of rules
                    for left_candidate in left_candidates:
                        for right_candidate in right_candidates:
                            if set(left_candidate).issubset(set(right_candidate)) or set(right_candidate).issubset(set(left_candidate)):
                                continue
                            right_candidate = tuple(set(right_candidate) - set(left_candidate))
                            # Given left_candidate, if right_candidate is superset of a 'past right_candidate' with confidence value smaller than required value,
                            # then its confidence value must be smaller than required value
                            if self.is_super_set(set(right_candidate), hist[left_candidate]):
                                hist[left_candidate].add(right_candidate)
                                continue
                            else:
                                confidence_val = self.get_confidence(data, set(left_candidate), set(right_candidate))
                                support = self.get_set_count(data, set(left_candidate).union(set(right_candidate)))/l
                                if confidence_val >= self.min_confidence:
                                    str_output = '{} -> {}'.format(left_candidate, right_candidate)
                                    if str_output not in res:
                                        print('Rule: {}  Support: {}  Confidence: {}'.format(str_output, support, confidence_val))
                                        res[str_output] = confidence_val
                                else:
                                    hist[left_candidate].add(right_candidate)
        return res
                        
                    
                    
                
            

### Read the data

In [3]:
df_all = pd.read_excel('./data.xlsx')
df_all.head()

,Username,Post_datetime,Content,Video_link(expired),Repost_Count,Comment_Count,Like_Count,Keywords,Video_file
0,Ice-dance柳鑫宇,2022-03-05 16:10:00.000000,我用了我很喜欢的一首歌曲来当背景音乐，来与大家分享我的闭幕式vlog，北京冬奥会真的结束了，...,https://f.video.weibocdn.com/o0/b7tMvrhxlx07Uf...,939,1532,29070,#冬奥会# #冬奥隔离日记#,./video/1.mp4
1,奥林匹克运动会,2022-03-16 12:30:00.000001,一起欣赏 @金博洋的天天 在北京2022年冬奥会上的短节目表演 #奥运会# ｜ #...,https://f.video.weibocdn.com/o0/Due4Y9s7lx07Uw...,852,305,2561,#奥运会# #北京2022年冬奥会#,./video/2.mp4
2,奥林匹克运动会,2022-03-18 08:00:00.000001,一起回顾 #羽生结弦# 的冬奥之旅 #奥运会# ｜ #北京2022年冬奥会# L...,https://f.video.weibocdn.com/o0/sJtalMgylx07Uy...,3565,851,19006,#羽生结弦# #奥运会# #北京2022年冬奥会#,./video/3.mp4
3,Ice-dance柳鑫宇,2022-02-23 14:21:00.000001,隔离中也要运动起来哦！ 训练计划安排！ 大家快跟我一起操练起来吧！ #冬奥会# #冬奥...,https://f.video.weibocdn.com/o0/DVYNperQlx07TY...,285,1009,15334,#冬奥会# #冬奥隔离日记#,./video/4.mp4
4,sy王诗玥,2022-02-23 14:21:00.000001,隔离期间也要动起来～ 大家跟我一起锻炼吧～🤸🏻‍♀️ #冬奥会# #冬奥隔离日记# ...,https://f.video.weibocdn.com/o0/E5AJHWkMlx07TY...,288,978,13871,#冬奥会# #冬奥隔离日记#,./video/5.mp4


### Preprocess the data

In [4]:
# Identify frequent phrases of differene length
def n_words_count(data, n):
        '''
        Return number of occurrence of words with length n.

        Parameters
        -----------
        data: List
            A list sentences or paragraphs
        
        n: int
            return the words with length n

        Returns
        ----------
        Dict:
            A dictionary mapping words with their frequency
        '''
        count_dict = defaultdict(lambda: 0)
        for sentence in data:
            
            for ptr1 in range(len(sentence)-n):
                ptr2 = ptr1 + n
                phrases = sentence[ptr1:ptr2]
                count_dict[phrases]+=1
        pairs = list(count_dict.items())
        pairs = sorted(pairs, key=lambda x: x[1])[::-1]
        return pairs
        

In [5]:
# Output frequency of word phrases with length 2 to 10
# Used to select meaningful frequent word phrase
data = list(df_all['Content'].values)
unwanted = re.compile('[!，?#：（）。《》、@……&“”！【】｜....____↓？ ]')
data = [unwanted.sub('', s) for s in data]
for word_l in range(2, 11):
    words_count = n_words_count(data, word_l)
    df = pd.DataFrame(words_count, columns=['words','count'])
    df.to_excel('./data/word_count_tables/wc{}.xlsx'.format(word_l), index=False)

In [6]:
freq_words = pd.read_excel(r'./data/freq_words.xlsx')
freq_words.head()

,words,count
0,残奥,991
1,开幕,265
2,跳台滑雪,116
3,金牌,183
4,闭幕,179


In [7]:
# Turn each post into a bag of features so that it can be passed to Apriori 
data = []
for idx, row in df_all.iterrows():
    bag = set()
    bag.add('Username: '+row['Username'])
    if not row['Keywords']:
        keywords = {kw for kw in row['Keywords'].split()}
        bag.update(keywords)
    text = row['Content']
    for interested_word in freq_words['words']:
        if interested_word in text:
            bag.add(interested_word)
    data.append(bag)
print(data[:5])

[{'Username: Ice-dance柳鑫宇', '闭幕'}, {'Username: 奥林匹克运动会'}, {'Username: 奥林匹克运动会', '羽生结弦'}, {'Username: Ice-dance柳鑫宇'}, {'Username: sy王诗玥'}]


### Apriori Algorithm

In [8]:
model = apriori(min_support=0.02, min_confidence=0.6)
rules = model.find_rules(data)

Rule: ('冰立方',) -> ('冰壶',)  Support: 0.02185792349726776  Confidence: 0.7058823529411765
Rule: ('雪车雪橇',) -> ('延庆',)  Support: 0.026411657559198543  Confidence: 0.6444444444444445
Rule: ('短道速滑',) -> ('Username: 北京2022年冬奥会',)  Support: 0.033697632058287796  Confidence: 0.6379310344827587
Rule: ('人民日报',) -> ('Username: 人民日报',)  Support: 0.02185792349726776  Confidence: 0.6
Rule: ('速度滑冰',) -> ('Username: 北京2022年冬奥会',)  Support: 0.03278688524590164  Confidence: 0.6792452830188679
Rule: ('冰丝带',) -> ('速度滑冰',)  Support: 0.02185792349726776  Confidence: 0.6666666666666666
Rule: ('闭幕',) -> ('残奥',)  Support: 0.05009107468123861  Confidence: 0.6179775280898876
Rule: ('鸟巢',) -> ('开幕',)  Support: 0.030054644808743168  Confidence: 0.7333333333333333
Rule: ('Username: 央视频',) -> ('残奥',)  Support: 0.033697632058287796  Confidence: 0.6379310344827587
Rule: ('雪容融',) -> ('残奥',)  Support: 0.061930783242258654  Confidence: 0.7816091954022989
Rule: ('冰壶',) -> ('残奥',)  Support: 0.04918032786885246  Confidence: 

### Result

Although not all rules are meaningful, some contain interesting facts. For example:

Rule: ('雪容融',) -> ('残奥',)  Support: 0.061930783242258654  Confidence: 0.7816091954022989

This rule relates the name of mascot ('雪容融') to Winter Paralympic Games ('残奥'). The confidence value tells that given that the word '雪容融' appears in the post content, there is around 78.16% of chance that the word '残奥' will also appear in the same post.